<strong><b><font size="5">Furto de Veiculos na Cidade de São Paulo - Parte 1</font></b></strong>

O objetivo dessa análise é verificar a proximidade de ocorrências de furtos de veículos com unidades de segurança na cidade de São Paulo
A ideia principal é que esse projeto possa ser uma ferramenta para ajudar os profissionais da justiça a visualizar de forma mais intuitiva como estão distribuídos as ocorrências de furtos na cidade.
O gráfico pode também dar informações importantes para uma alocação mais ampla de recursos para a aplicação da lei. Por exemplo, os dados podem ser usados para ajudar a determinar quais bairros ou regiões necessitam de mais policiamento com base nas áreas que ocorrem mais crimes.<br>
<b>A Análise é baseado nos meses de Março, Abril e Maio de 2021</b>
A Primeira parte do projeto vai fazer a análise exploratória e limpeza dos dados, na segunda parte será feita a plotagem dos dados no mapa.

![title](logoprojeto.png)

### Definição do Problema
Através de um conjunto de dados disponibilizados pelo portal de transparência da Secretaria de Segurança Pública do Estado São Paulo deve-se se mostrar num mapa a disposição das ocorrências de furto juntamente com a localização das principais unidades de segurança: Postos da Policia Militar, Bombeiros, Delegacias da Policia Civil e Guarda Civil Metropolitana.


### Fontes de Dados
Os dados podem ser baixados no link http://www.ssp.sp.gov.br/transparenciassp/ ao clicar na aba "Furto de Veículos"
O dicionário de dados (METODOLOGIA.docx) pode ser baixado ao clicar no botão "Exportar metodologia".
As localizações das unidades se segurança podem ser baixadas no site http://geosampa.prefeitura.sp.gov.br/
Pelo link dados abertos, é possível baixar os meta-dados que compõem o mapa, bastando filtrar por "Equipamentos de segurança", pode-se baixar um zip com uma série de arquivos sendo os que nos interessam são os do tipo ".DBF"


### Carregando os Pacotes Usados Neste Jupyter Notebook

In [1]:
# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

In [2]:
# As novas versões do Pandas e Matplotlib trazem diversas mensagens de aviso ao desenvolvedor. Vamos desativar isso.
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=matplotlib.cbook.mplDeprecation)

# Imports para manipulação de dados
import re
import json
import time
import numpy as np
import pandas as pd

In [3]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Celso Alves" --iversions

Author: Celso Alves

sys       : 3.7.6 (default, Jan  8 2020, 20:23:39) [MSC v.1916 64 bit (AMD64)]
matplotlib: 3.1.3
pandas    : 1.0.1
re        : 2.2.1
json      : 2.0.9
numpy     : 1.18.1



### Extração e Limpeza dos Dados - Data Cleaning
Ao clicar no botão "Exportar" é gerado um arquivo XLS

Importar o arquivo num dataframe Pandas

In [ ]:
df_furtos_maio_2021 = pd.read_excel('dados/DadosBO_2021_3(FURTO DE VEÍCULOS).xls')

Já no começo da analise o pandas gera o seguinte erro:<br>
<b>XLRDError: Unsupported format, or corrupt file: Expected BOF record; found b'A\x00N\x00O\x00_\x00'<b>



Tem uma pegadinha, ao baixar o arquivo do site, o mesmo é salvo com a extensão .xls, porém ao abri-lo no excel aparentimente não há problemas. Eu só descobri quando abri no notepad++, aí percebi que era um arquivo .CSV :(

Renomear as extensões dos arquivos para fazer mais sentido com seus conteúdos.
É necessário verificar tambem a codificação, deve-se converter para UFT8

In [4]:
#atenção aos parametros da função read_csv do pandas. Esse conjunto de dados tem como separador o tab(sep=t)
#é necessário verificar a codificação, por padrão vem ucs2-litle endian. Somente com os parametros default vai gerar erro no carregamento
#lineterminator='\r'
df_furtos_maio_2021 = pd.read_csv('dados/DadosBO_2021_5(FURTO DE VEÍCULOS).csv',sep='\t' )
df_furtos_abril_2021 = pd.read_csv('dados/DadosBO_2021_4(FURTO DE VEÍCULOS).csv',sep='\t' )
df_furtos_marco_2021 = pd.read_csv('dados/DadosBO_2021_3(FURTO DE VEÍCULOS).csv',sep='\t' )



Vou acrescentar uma coluna para diferenciar o periodo de cada dataset, assim fica mais facil diferenciar o periodo no mapa.

In [5]:
df_furtos_maio_2021['periodo'] = 'maio-2021'
df_furtos_abril_2021['periodo'] = 'abril-2021'
df_furtos_marco_2021['periodo'] = 'marco-2021'

Juntar os 3 dataframes em 1 e iniciar a analise e transformação dos dados


In [6]:
 df_furtos = df_furtos_maio_2021.append([df_furtos_abril_2021, df_furtos_marco_2021])

In [ ]:
# Shape
df_furtos.shape

In [7]:
#mostrar todas as colunas
pd.set_option('display.max_columns',None)

In [ ]:
# Visualizamos uma amostra de dados
df_furtos.head()

In [ ]:
#tipos de dados
df_furtos.info()

No conjunto de dados, algumas variaveis estão em branco, não faz sentido mante-las.


In [8]:
#dropna com parametro how='ALL', e axis1, somente se todas as linhas estiverem vazias ou null
df_furtos = df_furtos.dropna(how='all', axis=1)

In [ ]:
df_furtos.info()

Ok, reduzido de 58 para 50 colunas.

In [ ]:
df_furtos.info()

Para essa analise deve-se considerar apenas os registros da cidade de São Paulo.
Apagar os registros que tenham CIDADE como null para classificação e posterior filtragem.

In [9]:
#tem algum valor nulo na variavel cidade?
df_furtos['CIDADE'].isnull().sum()

8

In [10]:
#Apagar os dados em branco da cidade
df_furtos = df_furtos.dropna(subset=['CIDADE'])

In [11]:
#Verificando se ficou algum nulo
df_furtos[df_furtos['CIDADE'].isna()]


,ANO_BO,NUM_BO,NUMERO_BOLETIM,BO_INICIADO,BO_EMITIDO,DATAOCORRENCIA,HORAOCORRENCIA,PERIDOOCORRENCIA,DATACOMUNICACAO,DATAELABORACAO,BO_AUTORIA,FLAGRANTE,NUMERO_BOLETIM_PRINCIPAL,LOGRADOURO,NUMERO,BAIRRO,CIDADE,UF,LATITUDE,LONGITUDE,DESCRICAOLOCAL,EXAME,SOLUCAO,DELEGACIA_NOME,DELEGACIA_CIRCUNSCRICAO,ESPECIE,RUBRICA,DESDOBRAMENTO,STATUS,TIPOPESSOA,VITIMAFATAL,NATURALIDADE,SEXO,DATANASCIMENTO,IDADE,ESTADOCIVIL,PROFISSAO,GRAUINSTRUCAO,CORCUTIS,NATUREZAVINCULADA,TIPOVINCULO,PLACA_VEICULO,UF_VEICULO,CIDADE_VEICULO,DESCR_COR_VEICULO,DESCR_MARCA_VEICULO,ANO_FABRICACAO,ANO_MODELO,DESCR_TIPO_VEICULO,periodo


Ok, nenhum registro em branco da cidade

Analise do campo STATUS: FLAG_STATUS	Indica se é crime consumado ou tentado
Pra esse analise importa somente se o crime foi consumado.

In [12]:
df_furtos['STATUS'].value_counts()

Consumado    24080
Tentado         45
Name: STATUS, dtype: int64

In [13]:
#Remover os registros do tipo: 'TENTADO'
# Guardar os indexes onde status= tentado
indexNames = df_furtos[df_furtos['STATUS'] == 'Tentado'].index
# Apagar os indexes do dataframe
df_furtos.drop(indexNames , inplace=True)

In [14]:
df_furtos['STATUS'].value_counts()

Consumado    23991
Name: STATUS, dtype: int64

In [15]:
#Agora deve-se filtrar apenas a cidade de São paulo
df_furtos = df_furtos[df_furtos['CIDADE'].str.contains('(S.PAULO)')]

In [16]:
df_furtos['CIDADE'].value_counts()

S.PAULO    10071
Name: CIDADE, dtype: int64

In [17]:
#Analize da variavel DESCR_TIPO_VEICULO, que indica qual o tipo de veiculo foi roubado
df_furtos['DESCR_TIPO_VEICULO'].value_counts()

AUTOMOVEL          6657
MOTOCICLO          1979
CAMINHONETE         166
MOTONETA            113
CAMIONETA           106
Não Informado        97
CAMINHÃO             40
INEXIST.             40
UTILITÁRIO           12
MICRO-ONIBUS          9
REBOQUE               3
ONIBUS                2
CAMINHÃO TRATOR       1
SEMI-REBOQUE          1
CICLOMOTO             1
Name: DESCR_TIPO_VEICULO, dtype: int64

In [18]:
#tem algum valor nulo na variavel?
df_furtos['DESCR_TIPO_VEICULO'].isnull().sum()

844

In [19]:
#844 registros estão em branco, vou apagar pois é necessário esses valores para analise.
df_furtos = df_furtos.dropna(subset=['DESCR_TIPO_VEICULO'])

In [20]:
df_furtos['DESCR_TIPO_VEICULO'].isnull().sum()

0

Ok, Mais alguns ajustes baseado nas informações do dicionario de dados: 
"4. Para conclusões quanto as quantidades de ocorrências é necessária exclusão das duplicidades por meio dos campos: • NOME_DELEGACIA • ANO_BO • NUM_BO"


In [21]:
#ordenar os valores pelas variaveis informadas no dicionario de dados
df_furtos.sort_values(by=['DELEGACIA_NOME', 'ANO_BO','NUM_BO'], inplace = True)

In [22]:
#Apagar todos os os registros duplicados 
df_furtos.drop_duplicates(subset=['DELEGACIA_NOME', 'ANO_BO','NUM_BO'],
                     keep = 'first', inplace = True)

In [ ]:
#Vamos ver como está
df_furtos.sort_values(by=['NUM_BO'], inplace = True)
df_furtos.head(5)

In [23]:
df_furtos['periodo'].value_counts()

maio-2021     3025
marco-2021    2911
abril-2021    2762
Name: periodo, dtype: int64

<b>Variáveis que representam geolocalização</b><br>
As variáveis Latitude e Longitude representam as localizações das ocorrencias, são importantes para poder mostrar no mapa.


In [24]:
#Variaveis que representam a localização    
df_furtos[df_furtos['LATITUDE'].isna()]

,ANO_BO,NUM_BO,NUMERO_BOLETIM,BO_INICIADO,BO_EMITIDO,DATAOCORRENCIA,HORAOCORRENCIA,PERIDOOCORRENCIA,DATACOMUNICACAO,DATAELABORACAO,BO_AUTORIA,FLAGRANTE,NUMERO_BOLETIM_PRINCIPAL,LOGRADOURO,NUMERO,BAIRRO,CIDADE,UF,LATITUDE,LONGITUDE,DESCRICAOLOCAL,EXAME,SOLUCAO,DELEGACIA_NOME,DELEGACIA_CIRCUNSCRICAO,ESPECIE,RUBRICA,DESDOBRAMENTO,STATUS,TIPOPESSOA,VITIMAFATAL,NATURALIDADE,SEXO,DATANASCIMENTO,IDADE,ESTADOCIVIL,PROFISSAO,GRAUINSTRUCAO,CORCUTIS,NATUREZAVINCULADA,TIPOVINCULO,PLACA_VEICULO,UF_VEICULO,CIDADE_VEICULO,DESCR_COR_VEICULO,DESCR_MARCA_VEICULO,ANO_FABRICACAO,ANO_MODELO,DESCR_TIPO_VEICULO,periodo
4032,2021,1273,1273/2021,18/04/2021 09:49:44,18/04/2021 10:16:11,18/04/2021,02:00,DE MADRUGADA,18/04/2021,18/04/2021 09:49:44,Desconhecida,Não,NaN,NaN,0.0,BOM RETIRO,S.PAULO,SP,NaN,NaN,Via pública,NaN,ENCAMINHAMENTO DP ÁREA DO FATO,02º D.P. BOM RETIRO,12º D.P. PARI,Título II - Patrimônio (arts. 155 a 183),Furto (art. 155) - VEICULO,NaN,Consumado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,QUR7G67,SP,S.PAULO,Cinza,VW/GOL 1.0L MC4,2019.0,2020.0,AUTOMOVEL,abril-2021
348,2021,550,550/2021,02/03/2021 09:14:50,02/03/2021 09:25:41,05/12/2020,NaN,EM HORA INCERTA,02/03/2021,02/03/2021 09:14:50,Desconhecida,Não,NaN,NaN,0.0,ERMELINO MATARAZZO,S.PAULO,SP,NaN,NaN,Residência,NaN,ENCAMINHAMENTO DP ÁREA DO FATO,05º D.P. ACLIMACAO,62º D.P. ERMELINO MATARAZZO,Título II - Patrimônio (arts. 155 a 183),Furto (art. 155) - VEICULO,NaN,Consumado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DZP6580,SP,S.PAULO,Azul,HONDA/CG 150 TITAN KS,2008.0,2008.0,MOTOCICLO,marco-2021
6280,2021,889,889/2021,26/04/2021 18:27:46,26/04/2021 18:43:13,25/04/2021,NaN,EM HORA INCERTA,26/04/2021,26/04/2021 18:27:46,Desconhecida,Não,NaN,NaN,0.0,GRAJAU,S.PAULO,SP,NaN,NaN,Residência,NaN,ENCAMINHAMENTO DP ÁREA DO FATO,07º D.P. LAPA,85º D.P. JARDIM MIRNA,Título II - Patrimônio (arts. 155 a 183),Furto (art. 155) - VEICULO,NaN,Consumado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DUF9286,SP,SANTO ANDRE,Prata,RENAULT/CLIO AUT 10 H 3P,2006.0,2006.0,AUTOMOVEL,abril-2021
404,2021,951,951/2021,03/05/2021 11:54:28,03/05/2021 12:11:26,30/04/2021,NaN,EM HORA INCERTA,03/05/2021,03/05/2021 11:54:28,Desconhecida,Não,NaN,NaN,0.0,PERDIZES,S.PAULO,SP,NaN,NaN,Via pública,NaN,BO PARA INVESTIGAÇÃO,07º D.P. LAPA,07º D.P. LAPA,Título II - Patrimônio (arts. 155 a 183),Furto (art. 155) - VEICULO,NaN,Consumado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,QXQ8C69,MG,BELO HORIZONTE,Prata,HYUNDAI/HB20 1.0M 1.0 M,2020.0,2020.0,AUTOMOVEL,maio-2021
3153,2021,1960,1960/2021,14/04/2021 15:02:31,14/04/2021 15:43:54,11/04/2021,23:00,A NOITE,14/04/2021,14/04/2021 15:02:31,Desconhecida,Não,NaN,NaN,0.0,ITAQUERA,S.PAULO,SP,NaN,NaN,Residência,NaN,ENCAMINHAMENTO DP ÁREA DO FATO,08º D.P. BRAS,32º D.P. ITAQUERA,Título II - Patrimônio (arts. 155 a 183),"Furto qualificado (art. 155, §4o.) - VEICULO",NaN,Consumado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EAJ1199,SP,S.PAULO,Vermelho,YAMAHA/NMAX,2019.0,2019.0,MOTOCICLO,abril-2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8205,2021,958517,958517/2021,31/05/2021 18:38:24,31/05/2021 18:38:45,31/05/2021,12:00,PELA MANHÃ,31/05/2021,31/05/2021 18:38:24,Desconhecida,Não,NaN,AVENIDA DANTON JOBIM,569.0,SOCORRO,S.PAULO,SP,NaN,NaN,Via Pública,NaN,BO PARA INVESTIGAÇÃO,DELEGACIA ELETRONICA,102º D.P. SOCORRO,Título II - Patrimônio (arts. 155 a 183),Furto (art. 155) - VEICULO,NaN,Consumado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BLU1387,SP,SÃO PAULO,Prata,VW/PARATI CL,1993.0,NaN,AUTOMOVEL,maio-2021
8242,2021,959260,959260/2021,31/05/2021 20:54:20,31/05/2021 21:11:41,31/05/2021,17:50,A TARDE,31/05/2021,31/05/2021 20:54:20,Desconhecida,Não,NaN,RUA ARICANGA,919.0,VILA CURUCA,S.PAULO,SP,NaN,NaN,Via pública,NaN,ENCAMINHAMENTO DP ÁREA DO FATO,DELEGACIA ELETRONICA,59º D.P. JARDIM DOS IPES,Título II - Patrimônio (arts

943  linhas não tem latitude e longitude, sem esses valores não é possivel mostrar no mapa, vou apagar pois para essa analise a localidade é necessária

In [25]:
df_furtos = df_furtos.dropna(subset=['LATITUDE'])


In [27]:
#Informação do DF final
df_furtos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7755 entries, 2288 to 6069
Data columns (total 50 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ANO_BO                    7755 non-null   int64  
 1   NUM_BO                    7755 non-null   int64  
 2   NUMERO_BOLETIM            7755 non-null   object 
 3   BO_INICIADO               7755 non-null   object 
 4   BO_EMITIDO                7755 non-null   object 
 5   DATAOCORRENCIA            7755 non-null   object 
 6   HORAOCORRENCIA            7280 non-null   object 
 7   PERIDOOCORRENCIA          7755 non-null   object 
 8   DATACOMUNICACAO           7755 non-null   object 
 9   DATAELABORACAO            7755 non-null   object 
 10  BO_AUTORIA                7755 non-null   object 
 11  FLAGRANTE                 7755 non-null   object 
 12  NUMERO_BOLETIM_PRINCIPAL  173 non-null    object 
 13  LOGRADOURO                7755 non-null   object 
 14  NUMER

Temos 7755 registros de furtos, para essa analise esse quantidade é suficiente.

In [28]:
# Plotly
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.colors import n_colors
from plotly.subplots import make_subplots

# Supress warnings
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [29]:
#Mudar separador decimal de virgula(,) para ponto(.)
df_furtos['LATITUDE'] = df_furtos['LATITUDE'].str.replace(',', '.').astype(float)
df_furtos['LONGITUDE'] = df_furtos['LONGITUDE'].str.replace(',', '.').astype(float)


In [30]:
#visualizar 
#df_furtos[['LATITUDE','LONGITUDE']]

#df_furtos['LATITUDE'].isnull().sum()
df_furtos[df_furtos['LATITUDE'].isna()]

,ANO_BO,NUM_BO,NUMERO_BOLETIM,BO_INICIADO,BO_EMITIDO,DATAOCORRENCIA,HORAOCORRENCIA,PERIDOOCORRENCIA,DATACOMUNICACAO,DATAELABORACAO,BO_AUTORIA,FLAGRANTE,NUMERO_BOLETIM_PRINCIPAL,LOGRADOURO,NUMERO,BAIRRO,CIDADE,UF,LATITUDE,LONGITUDE,DESCRICAOLOCAL,EXAME,SOLUCAO,DELEGACIA_NOME,DELEGACIA_CIRCUNSCRICAO,ESPECIE,RUBRICA,DESDOBRAMENTO,STATUS,TIPOPESSOA,VITIMAFATAL,NATURALIDADE,SEXO,DATANASCIMENTO,IDADE,ESTADOCIVIL,PROFISSAO,GRAUINSTRUCAO,CORCUTIS,NATUREZAVINCULADA,TIPOVINCULO,PLACA_VEICULO,UF_VEICULO,CIDADE_VEICULO,DESCR_COR_VEICULO,DESCR_MARCA_VEICULO,ANO_FABRICACAO,ANO_MODELO,DESCR_TIPO_VEICULO,periodo


In [34]:
#salvar o df_furtos
df_furtos.to_csv('dados/df_furtos.csv', index = False,sep=';')


<b>Análise das Unidades de Segurança</b>

Vamos tentar incluir no mapa os pontos que representam unidades de segurança publica, tais como: delegacias, batalhões da policia miliar, guarda civil metropolitana, etc..
Os dados pode sem obtidos através do site http://geosampa.prefeitura.sp.gov.br/
Pelo link dados abertos , é possivel baixar os metadados que compoem o mapa, bastando filtrar por "Equipamentos de segurança", Vai baixar um zip com uma série de arquivos, os que nos interessam são os .DBF

Vamos carregar esses DBFs num Dataframe Pandas e analizar
Vou usar o pacote dbfread https://dbfread.readthedocs.io/en/latest/index.html

In [32]:
#instalando o pacote
!pip install dbfread

In [35]:
#Carregar o pacote
from dbfread import DBF
from pandas import DataFrame


In [39]:
#vamos abrir um 
dbf = DBF('dados/SEGURANCA_POLICIA_MILITAR.dbf')
df_unids_pm = pd.DataFrame(iter(dbf))



In [40]:
df_unids_pm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 189 entries, 0 to 188
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   eq_tema     189 non-null    object
 1   eq_esfera   189 non-null    object
 2   eq_classe   189 non-null    object
 3   eq_rede     189 non-null    object
 4   eq_tipo     189 non-null    object
 5   eq_nome     189 non-null    object
 6   eq_enderec  189 non-null    object
 7   eq_bairro   189 non-null    object
 8   eq_cep      189 non-null    object
 9   eq_cd_tema  189 non-null    object
 10  eq_cd_esfe  189 non-null    object
 11  eq_cd_clas  189 non-null    object
 12  eq_cd_rede  189 non-null    object
 13  eq_cd_tipo  189 non-null    object
 14  eq_id       189 non-null    object
dtypes: object(15)
memory usage: 22.3+ KB


In [41]:
df_unids_pm.head()

,eq_tema,eq_esfera,eq_classe,eq_rede,eq_tipo,eq_nome,eq_enderec,eq_bairro,eq_cep,eq_cd_tema,eq_cd_esfe,eq_cd_clas,eq_cd_rede,eq_cd_tipo,eq_id
0,SEGURANCA,SEM ESFERA,POLICIA MILITAR,SEM REDE,SEM TIPO,ACADEMIA DE POLÍCIA MILITAR DO BARRO BRANCO,AV AGUA FRIA 1923,ÁGUA FRIA,,7,0,5,0,0,150288
1,SEGURANCA,SEM ESFERA,POLICIA MILITAR,SEM REDE,SEM TIPO,ASSESSORIA DE POLÍCIA MILITAR DA ASSEMBLÉIA LE...,AV PEDRO ALVARES CABRAL 201,,,7,0,5,0,0,150289
2,SEGURANCA,SEM ESFERA,POLICIA MILITAR,SEM REDE,SEM TIPO,ASSESSORIA DE POLÍCIA MILITAR DA CÃMARA MUNICI...,R JACAREI 100,BELA VISTA,01319040,7,0,5,0,0,150290
3,SEGURANCA,SEM ESFERA,POLICIA MILITAR,SEM REDE,SEM TIPO,ASSESSORIA DE POLÍCIA MILITAR DO TRIBUNAL DE J...,R DR VILA NOVA 285,VILA BUARQUE,,7,0,5,0,0,150291
4,SEGURANCA,SEM ESFERA,POLICIA MILITAR,SEM REDE,SEM TIPO,ASSESSORIA POLICIAL MILITAR DA PROCURADORIA GE...,R RIACHUELO 115,SÉ,1007904,7,0,5,0,0,150292


In [42]:
#Para esse projeto interessa o campo endereço da unidade , vamos verificar se tem algum valor duplicado
df_unids_pm.sort_values(by=['eq_enderec'], inplace = True)

In [43]:
df_unids_pm['eq_enderec']

170        AV AFRANIO PEIXOTO  390
125         AV AFRANIO PEIXOTO 390
0                AV AGUA FRIA 1923
38               AV AGUA FRIA 1923
101           AV AIRTON PRETINI 69
                  ...             
127    R VISCONDE DE PARNAIBA 2334
9      R VISCONDE DE PARNAIBA 2334
172    R VISCONDE DE PARNAIBA 2334
12     R VISCONDE DE PARNAIBA 2334
149                    R ZURITA 98
Name: eq_enderec, Length: 189, dtype: object

In [44]:
#há espaços desnecessários nos endereços, por isso mesmo limpando as duplicidades, vai ficar algumas, vamos tratar
df_unids_pm['eq_enderec'] = df_unids_pm['eq_enderec'].str.replace(' +', ' ')
df_unids_pm['eq_enderec']


170         AV AFRANIO PEIXOTO 390
125         AV AFRANIO PEIXOTO 390
0                AV AGUA FRIA 1923
38               AV AGUA FRIA 1923
101           AV AIRTON PRETINI 69
                  ...             
127    R VISCONDE DE PARNAIBA 2334
9      R VISCONDE DE PARNAIBA 2334
172    R VISCONDE DE PARNAIBA 2334
12     R VISCONDE DE PARNAIBA 2334
149                    R ZURITA 98
Name: eq_enderec, Length: 189, dtype: object

In [45]:
#vamos remover os duplicados
df_unids_pm.sort_values(by=['eq_enderec'], inplace = True)
df_unids_pm.drop_duplicates(subset=['eq_enderec'],
                     keep = 'first', inplace = True)
df_unids_pm['eq_enderec']



170            AV AFRANIO PEIXOTO 390
0                   AV AGUA FRIA 1923
101              AV AIRTON PRETINI 69
184                  AV ANGELICA 1647
6             AV CARLOS OBERHUBER 397
                    ...              
29               R VICENTE AVELAR 100
143    R VILA OLIMPICA MARIO COVAS 21
50                       R VILELA 307
172       R VISCONDE DE PARNAIBA 2334
149                       R ZURITA 98
Name: eq_enderec, Length: 125, dtype: object

In [ ]:
pd.set_option('display.max_rows', 500)

<b>API de Geolocalização</b><br>
A API Google Geocodig é uma forma de utilizar um geocodificador através de um serviço HTTP. 
A geocodificação consiste em transformar endereços em coordenadas. Também é possível realizar a operação inversa, transformando coordenadas em endereços.
Esse serviço é utilizado normalmente para geocodificar endereços previamente conhecidos para que torne possível localizá-los em um mapa. 
Para esse projeto, como temos somente os endereços das unidades de segurança, vamos utilizar a API do google para trazer as coordenadas dos endereços e assim utilizar no mapa.


In [ ]:
# Instala o pacote googlemaps
!pip install -q googlemaps

In [47]:
# Importa o pacote
import googlemaps

In [ ]:
# Aqui definimos a API (coloque a sua chave) mais informações em 
https://developers.google.com/maps/documentation/geolocation/overview
gmaps_key = googlemaps.Client(key = 'SUA CHAVE ') 

In [49]:
# Listas para receber os dados
lista_latitude = []
lista_longitude = []

Evite executar esta célula abaixo muitas vezes para não bater o limite de requisições do Google Maps. Execute uma vez, faça os ajustes conforme mostrado nas células seguintes e salve o dataframe em disco.

In [ ]:
df_unids_pm[['eq_enderec', 'eq_bairro','eq_cep']][:2] 

In [ ]:
df_unids_pm['eq_enderec']

In [82]:
#função para buscar as coordenadas vi api geocode do google
def RetornaGeoCoordenadas(df):
    lista_latitude = []
    lista_longitude = []
    for label, row in df.iterrows():
        s = row['eq_enderec'] + ' ,' + row['eq_bairro'] + ' ,' + 'Sao Paulo - SP'
        print(s)
        # Obtém o geocode do endereço
        g = gmaps_key.geocode(s)
    
        # Coleta os dados de latitude e longitude
        # No caso de erro, atribui NaN
        try:
            lat = g[0]['geometry']['location']['lat']
            lng = g[0]['geometry']['location']['lng']
        except:
            lat = np.NaN
            lng = np.NaN
    
    # Grava nas listas
        lista_latitude.append(lat)
        lista_longitude.append(lng)
    return lista_latitude,lista_longitude




Loop para preencher as localizações

In [ ]:
for label, row in df_unids_pm.iterrows():
    s = row['eq_enderec'] + ' ,' + row['eq_bairro'] + ' ,' + 'Sao Paulo - SP'
    print(s)
 # Obtém o geocode do endereço
    g = gmaps_key.geocode(s)
    
    # Coleta os dados de latitude e longitude
    # No caso de erro, atribui NaN
    try:
        lat = g[0]['geometry']['location']['lat']
        lng = g[0]['geometry']['location']['lng']
    except:
        lat = np.NaN
        lng = np.NaN
    
    # Grava nas listas
    lista_latitude.append(lat)
    lista_longitude.append(lng)

In [51]:
# Agora adicionamos as colunas ao dataframe
df_unids_pm['latitude'] = lista_latitude
df_unids_pm['longitude'] = lista_longitude

In [52]:
df_unids_pm.head()

,eq_tema,eq_esfera,eq_classe,eq_rede,eq_tipo,eq_nome,eq_enderec,eq_bairro,eq_cep,eq_cd_tema,eq_cd_esfe,eq_cd_clas,eq_cd_rede,eq_cd_tipo,eq_id,latitude,longitude
170,SEGURANCA,SEM ESFERA,POLICIA MILITAR,SEM REDE,SEM TIPO,1º 6ºPELOTÃO DA 2ªCOMPANHIA DO 34ªBATALHÃO DA ...,AV AFRANIO PEIXOTO 390,BUTANTÃ,05507000,7,0,5,0,0,150458,-23.567232,-46.709141
0,SEGURANCA,SEM ESFERA,POLICIA MILITAR,SEM REDE,SEM TIPO,ACADEMIA DE POLÍCIA MILITAR DO BARRO BRANCO,AV AGUA FRIA 1923,ÁGUA FRIA,,7,0,5,0,0,150288,-23.472852,-46.619790
101,SEGURANCA,SEM ESFERA,POLICIA MILITAR,SEM REDE,SEM TIPO,5ªCOMPANHIA DE POLÍCIA MILITAR- - SÃO PAULO -...,AV AIRTON PRETINI 69,JARDIM AMÉRICA DA PENHA,03090000,7,0,5,0,0,150388,-23.529125,-46.554456
184,SEGURANCA,SEM ESFERA,POLICIA MILITAR,SEM REDE,SEM TIPO,SEDE DO 7º BPM/M,AV ANGELICA 1647,SANTA CECÍLIA,01301000,7,0,5,0,0,150472,-23.547012,-46.658066
6,SEGURANCA,SEM ESFERA,POLICIA MILITAR,SEM REDE,SEM TIPO,50°BATALHÃO DA POLÍCIA MILITAR - SÃO PAULO -...,AV CARLOS OBERHUBER 397,VILA SÃO JOSÉ,,7,0,5,0,0,150294,-23.741673,-46.709250


In [53]:
#salvar o df
# Salva o arquivo em disco (agora sem salvar o índice)
df_unids_pm.to_csv('dados/df_unid_PM.csv', index = False)


In [62]:
#ok, vamos fazer o mesmo com o DBF da Guarda Civil 
#vamos abrir um 
dbf = DBF('dados/GUARDA_CIVIL_METROPOLITANA.dbf')
df_unids_gcm = pd.DataFrame(iter(dbf))



In [63]:
df_unids_gcm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44 entries, 0 to 43
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   eq_tema     44 non-null     object
 1   eq_esfera   44 non-null     object
 2   eq_classe   44 non-null     object
 3   eq_rede     44 non-null     object
 4   eq_tipo     44 non-null     object
 5   eq_nome     44 non-null     object
 6   eq_enderec  44 non-null     object
 7   eq_bairro   44 non-null     object
 8   eq_cep      44 non-null     object
 9   eq_cd_tema  44 non-null     object
 10  eq_cd_esfe  44 non-null     object
 11  eq_cd_clas  44 non-null     object
 12  eq_cd_rede  44 non-null     object
 13  eq_cd_tipo  44 non-null     object
 14  eq_id       44 non-null     object
dtypes: object(15)
memory usage: 5.3+ KB


In [64]:
df_unids_gcm.head()

,eq_tema,eq_esfera,eq_classe,eq_rede,eq_tipo,eq_nome,eq_enderec,eq_bairro,eq_cep,eq_cd_tema,eq_cd_esfe,eq_cd_clas,eq_cd_rede,eq_cd_tipo,eq_id
0,SEGURANCA,SEM ESFERA,UNIDADE GUARDA CIVIL METROPOLITANA,SEM REDE,COMANDO OPERACIONAL CENTRO,COMANDO OPERACIONAL CENTRO,AV SANTOS DUMONT 767,BOM RETIRO,01101000,7,0,2,0,1,150477
1,SEGURANCA,SEM ESFERA,UNIDADE GUARDA CIVIL METROPOLITANA,SEM REDE,COMANDO OPERACIONAL NORTE,COMANDO OPERACIONAL NORTE,PC HERÓIS DA FEB,SANTANA,02022000,7,0,2,0,2,150478
2,SEGURANCA,SEM ESFERA,UNIDADE GUARDA CIVIL METROPOLITANA,SEM REDE,COMANDO OPERACIONAL SUL,COMANDO OPERACIONAL SUL,R CASSIANO DOS SANTOS 499,JARDIM CLIPPER,04827110,7,0,2,0,3,150479
3,SEGURANCA,SEM ESFERA,UNIDADE GUARDA CIVIL METROPOLITANA,SEM REDE,COMANDO OPERACIONAL LESTE,COMANDO OPERACIONAL LESTE,EST DE MOGI DAS CRUZES 1860,BURGO PAULISTA,03890100,7,0,2,0,4,150480
4,SEGURANCA,SEM ESFERA,UNIDADE GUARDA CIVIL METROPOLITANA,SEM REDE,COMANDO OPERACIONAL CENTRO-OESTE,COMANDO OPERACIONAL CENTRO-OESTE,AV PRFº FREDERICO HERMAN JUNIOR 653,PINHEIROS,05459010,7,0,2,0,5,150481


In [72]:
#para mim me interessa o campo endereço da unidade , vamos verificar se tem algum valor duplicado
df_unids_gcm.sort_values(by=['eq_enderec'], inplace = True)

In [73]:
df_unids_gcm['eq_enderec']

21                         AV DO ESTADO 680
23                         AV DO ESTADO 680
9                       AV DOS TEXTEIS 1331
17                  AV PEDRO ÁLVARES CABRAL
43                     AV PIRES DO RIO 1349
4       AV PRFº FREDERICO HERMAN JUNIOR 653
7            AV PROF JOAO BATISTA CONTI 285
31    AV RAIMUNDO PEREIRA DE MAGALHAES 5093
36                   AV ROBERT KENNEDY 2450
39                     AV SADAMU INOUE 6291
0                      AV SANTOS DUMONT 767
22                     AV SANTOS DUMONT 767
40                    AVENIDA MAL TITO 3012
3               EST DE MOGI DAS CRUZES 1860
41        EST VELHA DE MOGI DAS CRUZES 1860
6                         PC HAROLDO DALTRO
29                         PC HEROIS DA FEB
1                          PC HERÓIS DA FEB
12                       PC JOAO PISANI 449
24                             PC ROOSEVELT
11                   PC TANQUE DO ZUNEGA 31
30             R ADAUTO BEZERRA DELGADO 210
16             R BRENO FERRAZ DO

In [74]:
#há espaços desnecessários nos endereços, por isso mesmo limpando as duplicidades, vai ficar algumas, vamos tratar
df_unids_gcm['eq_enderec'] = df_unids_gcm['eq_enderec'].str.replace(' +', ' ')
df_unids_gcm['eq_enderec']

21                         AV DO ESTADO 680
23                         AV DO ESTADO 680
9                       AV DOS TEXTEIS 1331
17                  AV PEDRO ÁLVARES CABRAL
43                     AV PIRES DO RIO 1349
4       AV PRFº FREDERICO HERMAN JUNIOR 653
7            AV PROF JOAO BATISTA CONTI 285
31    AV RAIMUNDO PEREIRA DE MAGALHAES 5093
36                   AV ROBERT KENNEDY 2450
39                     AV SADAMU INOUE 6291
0                      AV SANTOS DUMONT 767
22                     AV SANTOS DUMONT 767
40                    AVENIDA MAL TITO 3012
3               EST DE MOGI DAS CRUZES 1860
41        EST VELHA DE MOGI DAS CRUZES 1860
6                         PC HAROLDO DALTRO
29                         PC HEROIS DA FEB
1                          PC HERÓIS DA FEB
12                       PC JOAO PISANI 449
24                             PC ROOSEVELT
11                   PC TANQUE DO ZUNEGA 31
30             R ADAUTO BEZERRA DELGADO 210
16             R BRENO FERRAZ DO

In [75]:
#vamos ver se tem duplicidades
df_unids_gcm['eq_enderec'].value_counts()


AV SANTOS DUMONT 767                     2
AV DO ESTADO 680                         2
AV PRFº FREDERICO HERMAN JUNIOR 653      1
EST DE MOGI DAS CRUZES 1860              1
R TAQUARI 635                            1
R MANOEL JOSE PEREIRA 300                1
R CAPITAO MACEDO 553                     1
R JOAO LUIZ CALHEIROS 40                 1
R ADAUTO BEZERRA DELGADO 210             1
R GAL COUTO DE MAGALHAES 444             1
R NOVA TUPAROQUERA 2220                  1
R FERNANDES PALERO 301                   1
PC ROOSEVELT                             1
R XIRO 266                               1
R DR FREDERICO HERMANN JUNIOR 653        1
PC TANQUE DO ZUNEGA 31                   1
AVENIDA MAL TITO 3012                    1
R CASSIANO DOS SANTOS 499                1
EST VELHA DE MOGI DAS CRUZES 1860        1
PC HAROLDO DALTRO                        1
R SEBASTIAO AFONSO 828                   1
VD DO CHA                                1
R ILIDIO DE FIGUEIREDO 0492              1
R DARWIN 22

In [76]:
#vamos remover os duplicados
df_unids_gcm.sort_values(by=['eq_enderec'], inplace = True)
df_unids_gcm.drop_duplicates(subset=['eq_enderec'],
                     keep = 'first', inplace = True)
df_unids_gcm['eq_enderec']

21                         AV DO ESTADO 680
9                       AV DOS TEXTEIS 1331
17                  AV PEDRO ÁLVARES CABRAL
43                     AV PIRES DO RIO 1349
4       AV PRFº FREDERICO HERMAN JUNIOR 653
7            AV PROF JOAO BATISTA CONTI 285
31    AV RAIMUNDO PEREIRA DE MAGALHAES 5093
36                   AV ROBERT KENNEDY 2450
39                     AV SADAMU INOUE 6291
22                     AV SANTOS DUMONT 767
40                    AVENIDA MAL TITO 3012
3               EST DE MOGI DAS CRUZES 1860
41        EST VELHA DE MOGI DAS CRUZES 1860
6                         PC HAROLDO DALTRO
29                         PC HEROIS DA FEB
1                          PC HERÓIS DA FEB
12                       PC JOAO PISANI 449
24                             PC ROOSEVELT
11                   PC TANQUE DO ZUNEGA 31
30             R ADAUTO BEZERRA DELGADO 210
16             R BRENO FERRAZ DO AMARAL 415
15                     R CAPITAO MACEDO 553
2                 R CASSIANO DOS

In [79]:
df_unids_gcm['eq_enderec'].value_counts()

R FERNANDES PALERO 301                   1
AV PRFº FREDERICO HERMAN JUNIOR 653      1
EST DE MOGI DAS CRUZES 1860              1
R TAQUARI 635                            1
R MANOEL JOSE PEREIRA 300                1
R CAPITAO MACEDO 553                     1
R JOAO LUIZ CALHEIROS 40                 1
R ADAUTO BEZERRA DELGADO 210             1
R GAL COUTO DE MAGALHAES 444             1
R NOVA TUPAROQUERA 2220                  1
AV SANTOS DUMONT 767                     1
PC ROOSEVELT                             1
R XIRO 266                               1
R DR FREDERICO HERMANN JUNIOR 653        1
PC TANQUE DO ZUNEGA 31                   1
AVENIDA MAL TITO 3012                    1
R CASSIANO DOS SANTOS 499                1
EST VELHA DE MOGI DAS CRUZES 1860        1
PC HAROLDO DALTRO                        1
AV DO ESTADO 680                         1
R SEBASTIAO AFONSO 828                   1
VD DO CHA                                1
R ILIDIO DE FIGUEIREDO 0492              1
R DARWIN 22

In [83]:
#ok, tiradas as duplicidades
#chamar a funcao para incluir as coordenadas
lista_lat,lista_long = RetornaGeoCoordenadas(df_unids_gcm)


AV DO ESTADO 680 ,BELA VISTA ,Sao Paulo - SP
AV DOS TEXTEIS 1331 ,SANTA ETELVINA ,Sao Paulo - SP
AV PEDRO ÁLVARES CABRAL ,IBIRAPUERA ,Sao Paulo - SP
AV PIRES DO RIO 1349 ,S. M. PAULISTA ,Sao Paulo - SP
AV PRFº FREDERICO HERMAN JUNIOR 653 ,PINHEIROS ,Sao Paulo - SP
AV PROF JOAO BATISTA CONTI 285 ,CONJUNTO JOSÉ BONIFÁCIO ,Sao Paulo - SP
AV RAIMUNDO PEREIRA DE MAGALHAES 5093 ,PIRITUBA ,Sao Paulo - SP
AV ROBERT KENNEDY 2450 ,JD. IPANEMA ,Sao Paulo - SP
AV SADAMU INOUE 6291 ,PARELHEIROS ,Sao Paulo - SP
AV SANTOS DUMONT 767 ,BOM RETIRO ,Sao Paulo - SP
AVENIDA MAL TITO 3012 ,ITAIM PAULISTA ,Sao Paulo - SP
EST DE MOGI DAS CRUZES 1860 ,BURGO PAULISTA ,Sao Paulo - SP
EST VELHA DE MOGI DAS CRUZES 1860 ,BURGO PAULISTA ,Sao Paulo - SP
PC HAROLDO DALTRO ,VILA MANCHESTER ,Sao Paulo - SP
PC HEROIS DA FEB ,SANTANA ,Sao Paulo - SP
PC HERÓIS DA FEB ,SANTANA ,Sao Paulo - SP
PC JOAO PISANI 449 ,BUTANTÃ ,Sao Paulo - SP
PC ROOSEVELT ,CONSOLAÇÃO ,Sao Paulo - SP
PC TANQUE DO ZUNEGA 31 ,JARDIM ROSELI ,Sao Paulo

In [84]:
df_unids_gcm['latitude'] = lista_lat
df_unids_gcm['longitude'] = lista_long

In [85]:
df_unids_gcm.head()

,eq_tema,eq_esfera,eq_classe,eq_rede,eq_tipo,eq_nome,eq_enderec,eq_bairro,eq_cep,eq_cd_tema,eq_cd_esfe,eq_cd_clas,eq_cd_rede,eq_cd_tipo,eq_id,latitude,longitude
21,SEGURANCA,SEM ESFERA,UNIDADE GUARDA CIVIL METROPOLITANA,SEM REDE,INSPETORIA REGIONAL AVENIDA PAULISTA,INSPETORIA REGIONAL AVENIDA PAULISTA,AV DO ESTADO 680,BELA VISTA,01315000,7,0,2,0,22,150498,-23.536137,-46.625642
9,SEGURANCA,SEM ESFERA,UNIDADE GUARDA CIVIL METROPOLITANA,SEM REDE,INSPETORIA REGIONAL CIDADE TIRADENTES,INSPETORIA REGIONAL CIDADE TIRADENTES,AV DOS TEXTEIS 1331,SANTA ETELVINA,08490600,7,0,2,0,10,150486,-23.592537,-46.403018
17,SEGURANCA,SEM ESFERA,UNIDADE GUARDA CIVIL METROPOLITANA,SEM REDE,INSPETORIA REGIONAL DO PARQUE IBIRAPUERA,INSPETORIA REGIONAL DO PARQUE IBIRAPUERA,AV PEDRO ÁLVARES CABRAL,IBIRAPUERA,04094050,7,0,2,0,18,150494,-23.580870,-46.659437
43,SEGURANCA,SEM ESFERA,UNIDADE GUARDA CIVIL METROPOLITANA,SEM REDE,INSPETORIA REGIONAL SÃO MIGUEL PAULISTA,INSPETORIA REGIONAL SÃO MIGUEL PAULISTA,AV PIRES DO RIO 1349,S. M. PAULISTA,0841000,7,0,2,0,44,150520,-23.508558,-46.443201
4,SEGURANCA,SEM ESFERA,UNIDADE GUARDA CIVIL METROPOLITANA,SEM REDE,COMANDO OPERACIONAL CENTRO-OESTE,COMANDO OPERACIONAL CENTRO-OESTE,AV PRFº FREDERICO HERMAN JUNIOR 653,PINHEIROS,05459010,7,0,2,0,5,150481,-23.562678,-46.703464


In [86]:
df_unids_gcm.to_csv('dados/df_unid_GCM.csv', index = False)

In [87]:
#Vou carregar o dbf da policia civil
dbf = DBF('dados/SEGURANCA_POLICIA_CIVIL.dbf')
df_unids_pcivil = pd.DataFrame(iter(dbf))


In [88]:
df_unids_pcivil.head()

,eq_tema,eq_esfera,eq_classe,eq_rede,eq_tipo,eq_nome,eq_enderec,eq_bairro,eq_cep,eq_cd_tema,eq_cd_esfe,eq_cd_clas,eq_cd_rede,eq_cd_tipo,eq_id
0,SEGURANCA,SEM ESFERA,POLICIA CIVIL,SEM REDE,SEM TIPO,DEATUR- DIVISÃO ESPECIAL DE ATENDIMENTO TURISTA,R SAO BENTO 380,CENTRO,,7,0,4,0,0,150135
1,SEGURANCA,SEM ESFERA,POLICIA CIVIL,SEM REDE,SEM TIPO,ASSISTENCIA POLICIAL DO DEMACRO,R PADRE DE CARVALHO 396,PINHEIROS,05427020,7,0,4,0,0,150098
2,SEGURANCA,SEM ESFERA,POLICIA CIVIL,SEM REDE,SEM TIPO,ACADEPOL - ACADEMIA DE POLÍCIA CIVIL DO ESTADO,PC PROF REYNALDO PORCHAT,BUTANTA,,7,0,4,0,0,150099
3,SEGURANCA,SEM ESFERA,POLICIA CIVIL,SEM REDE,SEM TIPO,APCS-ASSISTENCIA POL COMUNIC SOCIAL - DGPAD/DGP,R LIBERO BADARO 39,CENTRO,01009000,7,0,4,0,0,150100
4,SEGURANCA,SEM ESFERA,POLICIA CIVIL,SEM REDE,SEM TIPO,APCS-SECAO DE EXPEDIENTE /DGPAD/DGP,R LIBERO BADARO 39,CENTRO,01009000,7,0,4,0,0,150101


In [90]:
#há espaços desnecessários nos endereços, por isso mesmo limpando as duplicidades, vai ficar algumas, vamos tratar
df_unids_pcivil['eq_enderec'] = df_unids_pcivil['eq_enderec'].str.replace(' +', ' ')
df_unids_pcivil['eq_enderec']
#endereços duplicados
df_unids_pcivil['eq_enderec'].value_counts()

R LIBERO BADARO 39               19
R BRIGADEIRO TOBIAS 527           5
R PADRE DE CARVALHO 396           4
R MARQUES DE PARANAGUA 246        3
R DEPUTADO LACERDA FRANCO 372     3
                                 ..
R RIO BONITO 950                  1
R DA ESPERANCA 797                1
AV CONSELHEIRO CARRAO 2580        1
R SAPUCAIA 206                    1
R DA CANTAREIRA 390               1
Name: eq_enderec, Length: 125, dtype: int64

In [91]:
#vamos remover os duplicados
df_unids_pcivil.sort_values(by=['eq_enderec'], inplace = True)
df_unids_pcivil.drop_duplicates(subset=['eq_enderec'],
                     keep = 'first', inplace = True)
df_unids_pcivil['eq_enderec']

111                                  AL GLETE 827
151    AL PRIMEIRO SARGENTO OSMAR CORTEZ CLARO 52
131                          AV AIRTON PRETINI 69
176                             AV ALTANTICA 1171
95                      AV ANGELO CRISTIANINI 467
                          ...                    
136                R VISTOSA DA MADRE DE DEUS 120
149                    R. OTAVIO ALVES DUNDAS 390
70                       ROD DOS IMIGRANTES KM 11
36                                ROD HELIO SMIDT
39                              ZAKI NARCHIAV 152
Name: eq_enderec, Length: 125, dtype: object

In [92]:
#vou buscar as coordenadas de cada endereço
lista_lat,lista_long = RetornaGeoCoordenadas(df_unids_pcivil)

AL GLETE 827 ,CAMPOS ELISEOS ,Sao Paulo - SP
AL PRIMEIRO SARGENTO OSMAR CORTEZ CLARO 52 ,PARQUE NOVO MUNDO ,Sao Paulo - SP
AV AIRTON PRETINI 69 ,JARDIM  A PENHA ,Sao Paulo - SP
AV ALTANTICA 1171 ,SOCORRO ,Sao Paulo - SP
AV ANGELO CRISTIANINI 467 ,C JULIA ,Sao Paulo - SP
AV ARQUITETO VILA NOVA ARTIGAS 720 ,TEOTÔNIO VILELA ,Sao Paulo - SP
AV ARQUITETO VILANOVA ARTIGAS 720 ,TEOTÔNIO VILELA ,Sao Paulo - SP
AV ATLANTICA 1171 ,SOCORRO ,Sao Paulo - SP
AV CASA VERDE 677 ,CASA VERDE ,Sao Paulo - SP
AV CELSO GARCIA 2875 ,TATUAPE ,Sao Paulo - SP
AV CONSELHEIRO CARRAO 2580 ,VILA CARRAO ,Sao Paulo - SP
AV CORIFEU DE AZEVEDO MARQUES 4300 ,VILA LAGEADO ,Sao Paulo - SP
AV DEPUTADO EMILIO CARLOS 1251 ,LIMAO ,Sao Paulo - SP
AV DR GASTAO VIDIGAL 307 ,VILA LEOPOLDINA ,Sao Paulo - SP
AV ELISIO TEIXEIRA LEITE 7461 ,SIT MRO GRANDE ,Sao Paulo - SP
AV ENGENHEIRO GEORGE CORBISIER 322 ,JABAQUARA ,Sao Paulo - SP
AV ENGENHEIRO LUIZ CARLOS BERRINI 900 ,C MONCOES ,Sao Paulo - SP
AV INCONFIDENCIA MINEIRA 688 ,VILA RI

In [93]:
#guardar as coordendas em duas colunas no df
df_unids_pcivil['latitude'] = lista_lat
df_unids_pcivil['longitude'] = lista_long

In [94]:
df_unids_pcivil.head()

,eq_tema,eq_esfera,eq_classe,eq_rede,eq_tipo,eq_nome,eq_enderec,eq_bairro,eq_cep,eq_cd_tema,eq_cd_esfe,eq_cd_clas,eq_cd_rede,eq_cd_tipo,eq_id,latitude,longitude
111,SEGURANCA,SEM ESFERA,POLICIA CIVIL,SEM REDE,SEM TIPO,77º DISTRITO POLICIAL - SANTA CECILIA,AL GLETE 827,CAMPOS ELISEOS,01215001,7,0,4,0,0,150209,-23.536667,-46.648940
151,SEGURANCA,SEM ESFERA,POLICIA CIVIL,SEM REDE,SEM TIPO,90º DISTRITO POLICIAL DE SÃO PAULO,AL PRIMEIRO SARGENTO OSMAR CORTEZ CLARO 52,PARQUE NOVO MUNDO,,7,0,4,0,0,150249,-23.499688,-46.569125
131,SEGURANCA,SEM ESFERA,POLICIA CIVIL,SEM REDE,SEM TIPO,10º DISTRITO POLICIAL DA PENHA,AV AIRTON PRETINI 69,JARDIM A PENHA,03090000,7,0,4,0,0,150229,-23.529125,-46.554456
176,SEGURANCA,SEM ESFERA,POLICIA CIVIL,SEM REDE,SEM TIPO,6ª SECCIONAL/DECAP (SANTO AMARO),AV ALTANTICA 1171,SOCORRO,04768200,7,0,4,0,0,150274,-23.676329,-46.712057
95,SEGURANCA,SEM ESFERA,POLICIA CIVIL,SEM REDE,SEM TIPO,98º DISTRITO POLICIAL - JARDIM MIRIAM,AV ANGELO CRISTIANINI 467,C JULIA,04424010,7,0,4,0,0,150193,-23.685129,-46.637246


In [95]:
#Salvar o DF 
df_unids_pcivil.to_csv('dados/df_unid_pcivil.csv', index = False)

In [ ]:
Parte 1 finalizada - Proxima parte plotar os dados no grafico